In [12]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
cd ~/demres

/Users/zurfarosa/demres


In [14]:
import os
import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import pandas as pd
import numpy as np
from datetime import date, timedelta

import demres
from demres.common.constants import entry_type
from demres.demins.constants import Study_Design as sd
from demres.common import codelists
from demres.common.helper_functions import *
from demres.demins.statistical_functions import *

In [15]:
pd.set_option('display.max_columns', None)
pd.options.display.max_rows = 500

In [16]:
columns_for_inclusion = [
    'age_at_index_date',
    'female',
    'stroke',
    'heart_failure',
    'mental_illness',
    'sleep_apnoea', 
    'chronic_pulmonary_disease', 
    'hypnotics_100_pdds',
    'insomnia',
    'age_at_index_date:65-69', 'age_at_index_date:70-74',
    'age_at_index_date:75-79', 'age_at_index_date:80-84',
    'age_at_index_date:85-89', 'age_at_index_date:90-99',
    'age_at_index_date:above_99', 
    'hypnotics_100_pdds','hypnotic_pdds:00000',
    'hypnotic_pdds:00001_10', 'hypnotic_pdds:00011_100',
    'hypnotic_pdds:00101_1000', 'hypnotic_pdds:01001_10000',
    'hypnotic_pdds:10000_and_above'
]

In [33]:
for window in [sd.exposure_windows[1]]:
    printbold(window['name'])
    pt_features = pd.read_csv('data/processed_data/pt_features_avoid_specific_dementia_' + window['name'] +'.csv',delimiter=',',parse_dates=['index_date','data_end','data_start'],infer_datetime_format=True)
    print('Number of cases: ' + str(len(pt_features[pt_features['isCase']==True])))
#     display(pt_features.sample(3))
#     print(list(pt_features.columns))
    baselines = add_baseline_characteristics(columns_for_inclusion,pt_features)
    display(baselines[['Cases','Controls']])

**10_to_5**

Number of cases: 12879


,Cases,Controls
female,8552 (66.4%),8552 (66.4%)
stroke,1431 (11.1%),959 (7.4%)
heart_failure,764 (5.9%),618 (4.8%)
mental_illness,4766 (37.0%),3019 (23.4%)
sleep_apnoea,22 (0.2%),17 (0.1%)
chronic_pulmonary_disease,2944 (22.9%),2114 (16.4%)
insomnia,926 (7.2%),580 (4.5%)
age_at_index_date:65-69,452 (3.5%),452 (3.5%)
age_at_index_date:70-74,1161 (9.0%),1161 (9.0%)
age_at_index_date:75-79,2513 (19.5%),2513 (19.5%)
